In [1]:
import pandas as pd
import plac
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


In [2]:
data = pd.read_csv('./label_text_product_attrs.csv', dtype=str)

In [6]:
data.rename(columns={'sale_price': 'ppu', 'final_price': 'total_price'}, inplace=True)

In [7]:
data.iloc[0]

image_name                                        img_1_padded_0.jpg
text               04902430779746 NX DOWNY doahoa ngotngao\nVAT10...
sku                                                   04902430779746
product_name                               NX DOWNY doahoa ngotngao 
quantity                                                          1 
ppu                                                       88,000.00 
total_price                                               88,000.00 
discounted_part                                                  NaN
original_price                                           103,000.00 
Name: 0, dtype: object

In [191]:
def getNameProduct(text):
    start = 15
    end = 0
    productName = []
    label = 'product_name'
    for i in range(start,len(text)):
        if text[i]==' ' and text[i+1:i+7] == 'VAT10%':
            end_tmp = i
            productName.append((start,end_tmp,'product_name'))
            break
        if text[i] == ' ':
            end_tmp = i
            productName.append((start,end_tmp,'product_name'))
            start = end_tmp + 1
        
    return productName
    
def getQuantity(text, endNameProduct):
    start = 10000
    end = 0
    flag = 0
    for i in range(endNameProduct+7,len(text)):
        if text[i] != ' ' and flag == 0:
            start = i
            flag = 1
        if text[i] == ' ' and i > start:
            end = i
            break
    return start,end
    
def getPPU(text, endQuantity):
    start = 10000
    end = 0
    flag = 0
    for i in range(endQuantity,len(text)):
        if text[i] != ' ' and flag == 0:
            start = i
            flag = 1
        if text[i] == ' ' and i > start:
            end = i
            break
    return start,end
 
def getTotalPrice(text, endPPU):
    start = 10000
    end = 0
    flag = 0
    for i in range(endPPU,len(text)):
        if text[i] != ' ' and flag == 0:
            start = i
            flag = 1
        if text[i] == ' ' and i > start:
            end = i
            break
    return start,end

# def getOriginalPrice(text,endTP)
    
    
    
def get_row_entities(df):
    data = []
    for index, row in df.iterrows():
        entities = []
        text = row['text'].replace('\\n',' ')
        products = getNameProduct(text)
        endName = products[-1][1]
        startQuantity, endQuantity = getQuantity(text,endName)
        startPPU, endPPU = getPPU(text,endQuantity)
        startTP, endTP = getTotalPrice(text,endPPU)
#         startOriPrice,endOriPrice = getOriginalPrice(text,endTP)
        entities.append((0,14,'sku'))
        for product in products:
            entities.append(product)
        entities.append((startQuantity,endQuantity,'quantity'))
        entities.append((startPPU,endPPU,'ppu'))
        entities.append((startTP,endTP,'total_price'))
#         entities.append((startOriPrice,endOriPrice,'original_price'))
        res = (text,{"entities":entities})
        data.append(res)
    return data

In [152]:
str = '08936049051548'
len(str)

14

In [166]:
text = '04902430415637 NX DOWNY doahoa thom matcvd VAT10%   14214    129,900.00    129,421.00  Gia goc:   159,000.00 '
print(text)
a = getNameProduct(text)
# startQ, endQ = getQuantity(text,end)
# startPPU, endPPU = getPPU(text,endQ)
# startTP, endTP = getTotalPrice(text,endPPU)
for i in a:
    print(text[i[0]:i[1]])
print("Name",a)
# print("Quantity",text[startQ:endQ])
# print("PPU", text[startPPU:endPPU])
# print("TP", text[startTP: endTP])

04902430415637 NX DOWNY doahoa thom matcvd VAT10%   14214    129,900.00    129,421.00  Gia goc:   159,000.00 
NX
DOWNY
doahoa
thom
matcvd
Name [(15, 17, 'product_name'), (18, 23, 'product_name'), (24, 30, 'product_name'), (31, 35, 'product_name'), (36, 42, 'product_name')]


In [192]:
index = 15
tmp = get_row_entities(data)[index][0]
print(tmp)
for i in get_row_entities(data)[index][1]['entities']:
    print(f"{i[-1]}:",tmp[i[0]:i[1]])

08934679943325 Mi GAUDO vi tom & ga goi VAT10%   8    2,900.00    23,200.00  Gia goc:   3,200.00 
sku: 08934679943325
product_name: Mi
product_name: GAUDO
product_name: vi
product_name: tom
product_name: &
product_name: ga
product_name: goi
quantity: 8
ppu: 2,900.00
total_price: 23,200.00
